In [3]:
#Code to Retrieve Telecom Data from SQL and Summarize Customers by Contract Type

import pandas as pd
import sqlite3

# Load dataset
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(url)
print("Data imported successfully.")


conn = sqlite3.connect("customer_data.db")
df.to_sql("Telco_customers", conn, if_exists="replace", index=False)


full_df = pd.read_sql_query("SELECT * FROM Telco_customers", conn)


full_df['Churn'] = full_df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
full_df['TotalCharges'] = pd.to_numeric(full_df['TotalCharges'], errors='coerce')  # Convert TotalCharges to numeric
full_df['tenure'] = pd.to_numeric(full_df['tenure'], errors='coerce')  # Ensure tenure is numeric

# Generate summary statistics by contract type
contract_summary = full_df.groupby("Contract").agg(
    Total_Customers=("customerID", "count"),
    Churn_Rate_Percentage=("Churn", lambda x: round(x.mean() * 100, 2)),  # Convert to percentage
    Avg_Tenure_Months=("tenure", "mean"),
    Avg_Monthly_Charges=("MonthlyCharges", "mean"),
    Total_Revenue=("TotalCharges", "sum")
).reset_index()

contract_summary = contract_summary.round(2)

print(contract_summary)

conn.close()


Data imported successfully.
         Contract  Total_Customers  Churn_Rate_Percentage  Avg_Tenure_Months  \
0  Month-to-month             3875                  42.71              18.04   
1        One year             1473                  11.27              42.04   
2        Two year             1695                   2.83              56.74   

   Avg_Monthly_Charges  Total_Revenue  
0                66.40      5305861.5  
1                65.05      4467053.5  
2                60.77      6283253.7  
